In [ ]:
'''
Overview
this notebook parses all past ufc fight stats when run, it does not include upcoming fights


scrape ufc fight stats
get all event details, name, url, date, location for all ufc events
for each event, get fight details all fights on card
parse each fight to get fight stats of both fighters
'''

In [1]:
# imports
import pandas as pd
from tqdm.notebook import tqdm_notebook

# import library
import scrape_ufc_stats_library as LIB

# import config
import yaml
config = yaml.safe_load(open('scrape_ufc_stats_config.yaml'))

# Parse Event Details
Includes:
<br>
Event
<br>
URL
<br>
Date
<br>
Location
<br>

In [2]:
# define url to parse
events_url = config['completed_events_all_url']

In [3]:
# get soup
soup = LIB.get_soup(events_url)

# parse event details
all_event_details_df = LIB.parse_event_details(soup)

# show event details
display(all_event_details_df)

# write event details to file
all_event_details_df.to_csv(config['event_details_file_name'], index=False)

,EVENT,URL,DATE,LOCATION
0,UFC Fight Night: Tuivasa vs. Tybura,http://ufcstats.com/event-details/c398235fcaf8...,"March 16, 2024","Las Vegas, Nevada, USA"
1,UFC 299: O'Malley vs. Vera 2,http://ufcstats.com/event-details/a9df5ae20a97...,"March 09, 2024","Miami, Florida, USA"
2,UFC Fight Night: Rozenstruik vs. Gaziev,http://ufcstats.com/event-details/e4a9dbade7c7...,"March 02, 2024","Las Vegas, Nevada, USA"
3,UFC Fight Night: Moreno vs. Royval 2,http://ufcstats.com/event-details/902ab9197b83...,"February 24, 2024","Mexico City, Distrito Federal, Mexico"
4,UFC 298: Volkanovski vs. Topuria,http://ufcstats.com/event-details/dab0e6cb8c93...,"February 17, 2024","Anaheim, California, USA"
...,...,...,...,...
677,UFC 6: Clash of the Titans,http://ufcstats.com/event-details/1c3f5e85b59e...,"July 14, 1995","Casper, Wyoming, USA"
678,UFC 5: The Return of the Beast,http://ufcstats.com/event-details/dedc3bb440d0...,"April 07, 1995","Charlotte, North Carolina, USA"
679,UFC 4: Revenge of the Warriors,http://ufcstats.com/event-details/b60391da771d...,"December 16, 1994","Tulsa, Oklahoma, USA"
680,UFC 3: The American Dream,http://ufcstats.com/event-details/1a49e0670dfa...,"September 09, 1994","Charlotte, North Carolina, USA"


# Parse Fight Details
Includes:
<br>
Event
<br>
Bout
<br>
URL

In [4]:
# create empty df to store fight details
all_fight_details_df = pd.DataFrame(columns=config['fight_details_column_names'])

from tqdm import tqdm
# loop through each event and parse fight details
for index, row in tqdm(all_event_details_df.iterrows(), total=all_event_details_df.shape[0]):

    # get soup
    soup = LIB.get_soup(row['URL'])

    # parse fight links
    fight_details_df = LIB.parse_fight_details(soup)
    
    fight_details_df['DATE'] = row['DATE']
    fight_details_df['LOCATION'] = row['LOCATION']
    
    # concat fight details
    all_fight_details_df = pd.concat([all_fight_details_df, fight_details_df])

    all_fight_details_df = all_fight_details_df[['EVENT', 'BOUT', 'DATE', 'LOCATION', 'URL']]
    
# show all fight details
display(all_fight_details_df)

# write fight details to file
all_fight_details_df.to_csv(config['fight_details_file_name'], index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 682/682 [07:58<00:00,  1.42it/s]


,EVENT,BOUT,DATE,LOCATION,URL
0,UFC Fight Night: Tuivasa vs. Tybura,Marcin Tybura vs. Tai Tuivasa,"March 16, 2024","Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/8bc0c220f7dc...
1,UFC Fight Night: Tuivasa vs. Tybura,Bryan Battle vs. Ange Loosa,"March 16, 2024","Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/a7fd9da657bb...
2,UFC Fight Night: Tuivasa vs. Tybura,Ovince Saint Preux vs. Kennedy Nzechukwu,"March 16, 2024","Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/35f9beb94b4e...
3,UFC Fight Night: Tuivasa vs. Tybura,Christian Rodriguez vs. Isaac Dulgarian,"March 16, 2024","Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/acc7392d48cc...
4,UFC Fight Night: Tuivasa vs. Tybura,Macy Chiasson vs. Pannie Kianzad,"March 16, 2024","Las Vegas, Nevada, USA",http://ufcstats.com/fight-details/2f414494f2b6...
...,...,...,...,...,...
10,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,"March 11, 1994","Denver, Colorado, USA",http://ufcstats.com/fight-details/3b020d4914b4...
11,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,"March 11, 1994","Denver, Colorado, USA",http://ufcstats.com/fight-details/d917c8c7461b...
12,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,"March 11, 1994","Denver, Colorado, USA",http://ufcstats.com/fight-details/ccee020be2e8...
13,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,"March 11, 1994","Denver, Colorado, USA",http://ufcstats.com/fight-details/4b9ae533ccb3...


# Parse Fight Results and Fight Stats

Fight Results Includes:
<br>
Event
<br>
Bout
<br>
Weightclass
<br>
Method
<br>
Round
<br>
Time
<br>
Time Format
<br>
Referee
<br>
Details
<br>

Fight Stats Includes:
<br>
Event
<br>
Bout
<br>
Round
<br>
Fighter
<br>
Kd
<br>
Sig.Str.
<br>
Sig.Str. %
<br>
Total Str.
<br>
Td
<br>
Td %
<br>
Sub.Att
<br>
Rev.
<br>
Ctrl
<br>
Head
<br>
Body
<br>
Leg
<br>
Distance
<br>
Clinch
<br>
Ground
<br>

In [5]:
# create empty df to store fight results
all_fight_results_df = pd.DataFrame(columns=['EVENT', 'BOUT', 'OUTCOME', 'WEIGHTCLASS', 'METHOD', 'ROUND', 'TIME', 'TIME FORMAT', 'REFEREE', 'DETAILS', 'URL', 'DATE', 'LOCATION'])
# create empty df to store fight stats
all_fight_stats_df = pd.DataFrame(columns=['EVENT', 'BOUT', 'ROUND', 'FIGHTER', 'KD', 'SIG.STR.', 'SIG.STR. %', 'TOTAL STR.', 'TD', 'TD %', 'SUB.ATT', 'REV.', 'CTRL', 'HEAD', 'BODY', 'LEG', 'DISTANCE', 'CLINCH', 'GROUND', 'DATE', 'LOCATION'])

# loop through each fight and parse fight results and stats
for index, row in tqdm(all_fight_details_df.iterrows(), total=all_fight_details_df.shape[0]):

    # get soup
    soup = LIB.get_soup(row['URL'])

    # parse fight results and fight stats
    fight_results_df, fight_stats_df = LIB.parse_organise_fight_results_and_stats(
        soup,
        row['URL'],
        config['fight_results_column_names'],
        config['totals_column_names'],
        config['significant_strikes_column_names']
        )

    fight_results_df['DATE'] = row['DATE']
    fight_results_df['LOCATION'] = row['LOCATION']

    fight_stats_df['DATE'] = row['DATE']
    fight_stats_df['LOCATION'] = row['LOCATION']
    
    # concat fight results
    all_fight_results_df = pd.concat([all_fight_results_df, fight_results_df])
    # concat fight stats
    all_fight_stats_df = pd.concat([all_fight_stats_df, fight_stats_df])


# show all fight results
display(all_fight_results_df)
# show all fight stats
display(all_fight_stats_df)

# write to file
all_fight_results_df.to_csv(config['fight_results_file_name'], index=False)
# write to file
all_fight_stats_df.to_csv(config['fight_stats_file_name'], index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7569/7569 [2:10:01<00:00,  1.03s/it]


,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL,DATE,LOCATION
0,UFC Fight Night: Tuivasa vs. Tybura,Tai Tuivasa vs. Marcin Tybura,L/W,Heavyweight Bout,Submission,1,4:08,5 Rnd (5-5-5-5-5),Herb Dean,Rear Naked Choke,http://ufcstats.com/fight-details/8bc0c220f7dc...,"March 16, 2024","Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Bryan Battle vs. Ange Loosa,NC/NC,Welterweight Bout,Could Not Continue,2,1:00,3 Rnd (5-5-5),Mike Beltran,Eye injury to Loosa,http://ufcstats.com/fight-details/a7fd9da657bb...,"March 16, 2024","Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Ovince Saint Preux vs. Kennedy Nzechukwu,W/L,Light Heavyweight Bout,Decision - Split,3,5:00,3 Rnd (5-5-5),Mark Smith,Adalaide Byrd 28 - 29.Derek Cleary 29 - 28.Jun...,http://ufcstats.com/fight-details/35f9beb94b4e...,"March 16, 2024","Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Christian Rodriguez vs. Isaac Dulgarian,W/L,Featherweight Bout,Decision - Split,3,5:00,3 Rnd (5-5-5),Chris Tognoni,Sal D'amato 27 - 28.Anthony Maness 28 - 27.Ron...,http://ufcstats.com/fight-details/acc7392d48cc...,"March 16, 2024","Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Pannie Kianzad vs. Macy Chiasson,L/W,Women's Bantamweight Bout,Submission,1,3:54,3 Rnd (5-5-5),Mark Smith,Rear Naked Choke,http://ufcstats.com/fight-details/2f414494f2b6...,"March 16, 2024","Las Vegas, Nevada, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,W/L,Open Weight Bout,KO/TKO,1,2:50,No Time Limit,John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...,"March 11, 1994","Denver, Colorado, USA"
0,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,W/L,Open Weight Bout,Submission,1,4:52,No Time Limit,John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...,"March 11, 1994","Denver, Colorado, USA"
0,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,W/L,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...,"March 11, 1994","Denver, Colorado, USA"
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,W/L,Open Weight Bout,Submission,1,0:58,No Time Limit,John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...,"March 11, 1994","Denver, Colorado, USA"


,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,...,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND,DATE,LOCATION
0,UFC Fight Night: Tuivasa vs. Tybura,Tai Tuivasa vs. Marcin Tybura,Round 1,Tai Tuivasa,0,15 of 24,62%,18 of 27,0 of 0,---,...,0,0:00,9 of 17,1 of 2,5 of 5,8 of 17,7 of 7,0 of 0,"March 16, 2024","Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Tai Tuivasa vs. Marcin Tybura,Round 1,Marcin Tybura,0,27 of 41,65%,73 of 96,1 of 4,25%,...,0,2:48,23 of 34,4 of 7,0 of 0,8 of 20,0 of 0,19 of 21,"March 16, 2024","Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Bryan Battle vs. Ange Loosa,Round 1,Bryan Battle,0,23 of 63,36%,24 of 66,1 of 2,50%,...,0,1:33,12 of 45,10 of 17,1 of 1,20 of 57,3 of 5,0 of 1,"March 16, 2024","Las Vegas, Nevada, USA"
1,UFC Fight Night: Tuivasa vs. Tybura,Bryan Battle vs. Ange Loosa,Round 2,Bryan Battle,0,4 of 12,33%,4 of 12,0 of 0,---,...,0,0:00,4 of 11,0 of 1,0 of 0,4 of 12,0 of 0,0 of 0,"March 16, 2024","Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Bryan Battle vs. Ange Loosa,Round 1,Ange Loosa,0,17 of 44,38%,20 of 48,0 of 1,0%,...,0,0:00,4 of 29,6 of 6,7 of 9,16 of 43,1 of 1,0 of 0,"March 16, 2024","Las Vegas, Nevada, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,Round 1,David Levicki,0,4 of 5,80%,95 of 102,0 of 0,---,...,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,"March 11, 1994","Denver, Colorado, USA"
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Patrick Smith,0,1 of 1,100%,1 of 1,0 of 1,0%,...,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,"March 11, 1994","Denver, Colorado, USA"
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Ray Wizard,0,1 of 1,100%,2 of 2,0 of 0,---,...,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,"March 11, 1994","Denver, Colorado, USA"
0,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,Round 1,Scott Morris,0,1 of 1,100%,2 of 2,1 of 1,100%,...,0,--,1 of 1,0 of 0,0 of 0,0 of 0,1 of 1,0 of 0,"March 11, 1994","Denver, Colorado, USA"


In [9]:
# 誕生日を条件付きでdate型に変換（無効な日付はNaTになる）
all_fight_results_df['DATE'] = pd.to_datetime(all_fight_results_df['DATE'], errors='coerce')
all_fight_results_df
all_fight_results_df.to_csv(config['fight_results_file_name'], index=False)

all_fight_stats_df['DATE'] = pd.to_datetime(all_fight_stats_df['DATE'], errors='coerce')
all_fight_stats_df
all_fight_stats_df.to_csv(config['fight_stats_file_name'], index=False)

In [8]:
all_fight_stats_df['DATE'] = pd.to_datetime(all_fight_stats_df['DATE'], errors='coerce')
all_fight_stats_df

,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,...,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND,DATE,LOCATION
0,UFC Fight Night: Tuivasa vs. Tybura,Tai Tuivasa vs. Marcin Tybura,Round 1,Tai Tuivasa,0,15 of 24,62%,18 of 27,0 of 0,---,...,0,0:00,9 of 17,1 of 2,5 of 5,8 of 17,7 of 7,0 of 0,2024-03-16,"Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Tai Tuivasa vs. Marcin Tybura,Round 1,Marcin Tybura,0,27 of 41,65%,73 of 96,1 of 4,25%,...,0,2:48,23 of 34,4 of 7,0 of 0,8 of 20,0 of 0,19 of 21,2024-03-16,"Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Bryan Battle vs. Ange Loosa,Round 1,Bryan Battle,0,23 of 63,36%,24 of 66,1 of 2,50%,...,0,1:33,12 of 45,10 of 17,1 of 1,20 of 57,3 of 5,0 of 1,2024-03-16,"Las Vegas, Nevada, USA"
1,UFC Fight Night: Tuivasa vs. Tybura,Bryan Battle vs. Ange Loosa,Round 2,Bryan Battle,0,4 of 12,33%,4 of 12,0 of 0,---,...,0,0:00,4 of 11,0 of 1,0 of 0,4 of 12,0 of 0,0 of 0,2024-03-16,"Las Vegas, Nevada, USA"
0,UFC Fight Night: Tuivasa vs. Tybura,Bryan Battle vs. Ange Loosa,Round 1,Ange Loosa,0,17 of 44,38%,20 of 48,0 of 1,0%,...,0,0:00,4 of 29,6 of 6,7 of 9,16 of 43,1 of 1,0 of 0,2024-03-16,"Las Vegas, Nevada, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,Round 1,David Levicki,0,4 of 5,80%,95 of 102,0 of 0,---,...,0,--,4 of 5,0 of 0,0 of 0,1 of 2,2 of 2,1 of 1,1994-03-11,"Denver, Colorado, USA"
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Patrick Smith,0,1 of 1,100%,1 of 1,0 of 1,0%,...,0,--,0 of 0,1 of 1,0 of 0,0 of 0,1 of 1,0 of 0,1994-03-11,"Denver, Colorado, USA"
0,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Ray Wizard,0,1 of 1,100%,2 of 2,0 of 0,---,...,0,--,0 of 0,0 of 0,1 of 1,1 of 1,0 of 0,0 of 0,1994-03-11,"Denver, Colorado, USA"
0,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,Round 1,Scott Morris,0,1 of 1,100%,2 of 2,1 of 1,100%,...,0,--,1 of 1,0 of 0,0 of 0,0 of 0,1 of 1,0 of 0,1994-03-11,"Denver, Colorado, USA"
